In [5]:
#image compression 

In [6]:
import math as m
import numpy as np


In [23]:
test_blocks = np.asarray([[247, 250, 253, 255, 255, 254, 254, 255],
       [247, 250, 253, 255, 255, 254, 254, 255],
       [248, 250, 254, 255, 255, 254, 254, 255],
       [249, 251, 254, 255, 255, 254, 254, 255],
       [250, 252, 254, 255, 254, 254, 255, 255],
       [251, 253, 255, 255, 254, 254, 255, 255],
       [251, 253, 255, 255, 254, 254, 255, 255],
       [252, 253, 255, 255, 254, 254, 255, 255]])
test_blocks = test_blocks - 128

In [36]:
print(test_blocks)

[[119 122 125 127 127 126 126 127]
 [119 122 125 127 127 126 126 127]
 [120 122 126 127 127 126 126 127]
 [121 123 126 127 127 126 126 127]
 [122 124 126 127 126 126 127 127]
 [123 125 127 127 126 126 127 127]
 [123 125 127 127 126 126 127 127]
 [124 125 127 127 126 126 127 127]]


In [47]:
QM = np.asarray([[16 ,11 ,10 ,16 ,24 ,40 ,51 ,61],
       [12 ,12 ,14 ,19 ,26 ,58 ,60 ,55],
       [14 ,13 ,16 ,24 ,40 ,57 ,69 ,56],
       [14 ,17 ,22 ,29 ,51 ,87 ,80 ,62],
       [18 ,22 ,37 ,56 ,68 ,109 ,103 ,77],
       [24 ,35 ,55 ,64 ,81 ,104 ,113 ,92],
       [49 ,64 ,78 ,87 ,103 ,121 ,120 ,101],
       [72 ,92 ,95 ,98 ,112 ,100 ,103 ,99]])


def DCT_n_Quanta(block, QM = QM):
    N = block.shape[0]
    new_block = np.zeros(block.shape, dtype = 'int')
    output = np.zeros(block.shape, dtype = 'int')
    # print('test: ' + str(block.shape))
    for v in range(N):
        for u in range(N):
            temp = 0
            for x in range(N):
                for y in range(N):
                    a = m.cos((2*x +1)*u*m.pi / 16 )
                    b = m.cos((2*y +1)*v*m.pi / 16 )
                    temp = temp + (block[x,y]*a*b)
            Cv = m.sqrt(1/2) if v == 0 else 1
            Cu = m.sqrt(1/2) if u == 0 else 1
            new_block[v,u] = temp*Cv*Cu*(1/4)
           
            output[v,u] = new_block[v,u]//QM[v,u]
    print('new blocks')
    print(new_block)
    return output



In [48]:
DCT_test_blocks = DCT_n_Quanta(test_blocks)
print(DCT_test_blocks)

pair: 1004 n 16
pair: 1194 n 11
pair: 644 n 10
pair: 63 n 16
pair: -275 n 24
pair: -279 n 40
pair: -68 n 51
pair: 138 n 61
pair: 1193 n 12
pair: 1420 n 12
pair: 766 n 14
pair: 74 n 19
pair: -327 n 26
pair: -332 n 58
pair: -81 n 60
pair: 164 n 55
pair: 640 n 14
pair: 762 n 13
pair: 410 n 16
pair: 38 n 24
pair: -178 n 40
pair: -179 n 57
pair: -44 n 69
pair: 88 n 56
pair: 56 n 14
pair: 66 n 17
pair: 33 n 22
pair: 0 n 29
pair: -19 n 51
pair: -18 n 87
pair: -4 n 80
pair: 8 n 62
pair: -283 n 18
pair: -338 n 22
pair: -185 n 37
pair: -22 n 56
pair: 72 n 68
pair: 75 n 109
pair: 18 n 103
pair: -38 n 77
pair: -288 n 24
pair: -343 n 35
pair: -188 n 55
pair: -22 n 64
pair: 74 n 81
pair: 76 n 104
pair: 18 n 113
pair: -38 n 92
pair: -76 n 49
pair: -91 n 64
pair: -51 n 78
pair: -8 n 87
pair: 17 n 103
pair: 18 n 121
pair: 4 n 120
pair: -9 n 101
pair: 131 n 72
pair: 155 n 92
pair: 82 n 95
pair: 5 n 98
pair: -38 n 112
pair: -38 n 100
pair: -9 n 103
pair: 18 n 99
new blocks
[[1004 1194  644   63 -275 -279

In [40]:
def re_quanta(block, QM = QM):
    output = np.zeros(block.shape, dtype = 'int')
    for i in range(block.shape[0]):
        for j in range(block.shape[0]):
            output[i,j] = block[i,j]*QM[i,j]
    return output


In [42]:
temp_block = re_quanta(DCT_test_blocks)
print(temp_block)

[[ 992 1188  640   48 -288 -280 -102  122]
 [1188 1416  756   57 -338 -348 -120  110]
 [ 630  754  400   24 -200 -228  -69   56]
 [  56   51   22    0  -51  -87  -80    0]
 [-288 -352 -185  -56   68    0    0  -77]
 [-288 -350 -220  -64    0    0    0  -92]
 [ -98 -128  -78  -87    0    0    0 -101]
 [  72   92    0    0 -112 -100 -103    0]]


In [49]:
def iDCT(block):
    output = np.zeros(block.shape, dtype = 'int')
    for x in range(block.shape[0]):
        for y in range(block.shape[0]):
            temp = 0
            for v in range(block.shape[0]):
                for u in range(block.shape[0]):
                    a = m.cos( (2*x+1)*u*m.pi /16)
                    b = m.cos( (2*y+1)*v*m.pi /16)
                    Cv = m.sqrt(1/2) if v == 0 else 1
                    Cu = m.sqrt(1/2) if u == 0 else 1
                    temp = temp + a*b*Cu*Cv*block[v,u]
            output[x,y] = temp*(1/4)
    return output

In [50]:
final_block = iDCT(temp_block)
print(final_block)

[[1010 1441  602  -57   21   -5  -18    8]
 [1416 1379  737 -133   71  -50   49  -33]
 [ 625  752  381  -33   -3   -9   14    5]
 [-103 -134  -26   18  -25   23  -41    5]
 [  53   75   -5  -29   43  -22   34  -18]
 [ -25  -67  -12    7  -25  -10    6   -6]
 [  -4   52   15  -23   32    6   26  -21]
 [   9  -25  -21   29  -25   19  -46   26]]
